In [4]:
import os
os. chdir('/home/users/gmenon/workspace/songsLyricsGenerator/src')

In [54]:
import torch
from torch import nn
from pytorch_lightning import LightningModule
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model
from constants.mir_constants import TrainingArgs, WAV2VEC2_ARGS
from dataclasses import dataclass, asdict
import json
import argparse
from dataclasses import dataclass

In [6]:
from flash.audio import SpeechRecognitionData

In [8]:
print(json.dumps(asdict(WAV2VEC2_ARGS), indent = 4))

{
    "TRAIN_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_en_demucs_cleaned.csv",
    "TEST_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 1,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt",
    "FINETUNE_STRATEGY": "no_freeze_deepspeed",
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}


In [6]:
datamodule = SpeechRecognitionData.from_csv("consolidated_file_path",
                                                         "transcription_capitalized",
                                                         train_file=WAV2VEC2_ARGS.TRAIN_FILE_PATH,
                                                         test_file=WAV2VEC2_ARGS.TEST_FILE_PATH,
                                                         batch_size=WAV2VEC2_ARGS.BATCH_SIZE
                                                         )

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
datamodule

In [7]:
import flash
import torchaudio
import pytorch_lightning as pl

class SpeechDataModule(pl.LightningDataModule):
    def __init__(self, args: WAV2VEC2_ARGS):
        super().__init__()
        self.batch_size = 2

    def prepare_data(self):
        # Download and extract the audio files and transcripts, if necessary.
        pass

    def setup(self, stage=None):
        # Create the dataset
        # datamodule = 
        self.dataset = SpeechRecognitionData.from_csv("consolidated_file_path",
                                                         "transcription_capitalized",
                                                         train_file=WAV2VEC2_ARGS.TRAIN_FILE_PATH,
                                                         test_file=WAV2VEC2_ARGS.TEST_FILE_PATH,
                                                         batch_size=2
                                                         )

        # Create the dataloader
        self.train_dataloader = self.dataset.train_dataloader
        # torch.utils.data.DataLoader(
        #     self.dataset,
        #     batch_size=self.batch_size,
        #     shuffle=True,
        #     num_workers=4,
        # )
        self.val_dataloader = self.dataset.val_dataloader
        # torch.utils.data.DataLoader(
        #     self.dataset,
        #     batch_size=self.batch_size,
        #     shuffle=False,
        #     num_workers=4,
        # )

    def train_dataloader(self):
        return self.train_dataloader

    def val_dataloader(self):
        return self.val_dataloader

In [8]:
# class DALIDataset(pl.LightningDataModule):
#     def __init__(self, batch_size: int = 4,
#                   train_path :Optional[str] = None,
#                     validation_path: Optional[str] = None,
#                       model_backbone: pl.LightningModule = None,
#                       args: TrainingArgs = WAV2VEC2_ARGS
#                       ):
        
#         super().__init__()
#         self.train_path = train_path if train_path is not None else args.TRAIN_FILE_PATH
#         self.validation_path = validation_path if validation_path is not None else args.TEST_FILE_PATH
#         self.model_backbone = model_backbone if model_backbone is not None else args.MODEL_BACKBONE

#         def prepare_data(self):
#             pass
        
#         def setup(self):
#             train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH) 
#             validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
#             songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
#             audio_dataset = Dataset.from_dict(
#                 {"audio": list(songs_metadata["file_name"]),
#                  "transcription": list(songs_metadata["transcription"])}).cast_column("audio", Audio(sampling_rate=16_000))
#             audio_dataset["transcription"] = audio_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', audio_dataset["transcription"]).upper()
#             audio_dataset = audio_dataset.train_test_split(test_size=0.2, shuffle=True)
            



In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

class SpeechRecognitionDataset(Dataset):
    def __init__(self, dataset,args:TrainingArgs,processor):
        self.model_backbone = args.MODEL_BACKBONE
        self.dataset = dataset
        self.processor = processor
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(self.model_backbone)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(self.model_backbone)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        
    def __getitem__(self, index):
        audio = self.dataset[index]["audio"][0]
        transcription = self.dataset[index]["transcription"][0]

        input_values = self.processor(audio["array"], sampling_rate=16_000).input_values[0]
        with self.processor.as_target_processor():
            print("Entering the label encoder")
            labels = self.processor(transcription,return_tensors = 'pt').input_ids
       # attention_mask = inputs.attention_mask.squeeze()
        return input_values, labels
    
    def __len__(self):
        return len(self.dataset)



In [119]:
class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.cpu_count = 1
        print(f"CPU Count = {self.cpu_count}")
        self.padding: Union[bool, str] = True
        self.max_length: Optional[int] = None
        self.max_length_labels: Optional[int] = None
        self.pad_to_multiple_of: Optional[int] = None
        self.pad_to_multiple_of_labels: Optional[int] = None
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH).head(10)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(10)
        #songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

            
            train_dataset = train_dataset.map(self.transform_huggingface_dataset, num_proc = self.cpu_count)
            train_dataset =train_dataset.remove_columns(['audio','transcription','input_length'])
            self.train_dataset = train_dataset.map(self.data_collate, num_proc = self.cpu_count)

            
            val_dataset = val_dataset.map(self.transform_huggingface_dataset, num_proc = self.cpu_count,)
            val_dataset =val_dataset.remove_columns(['audio','transcription','input_length'])
            self.val_dataset = val_dataset.map(self.data_collate, num_proc = self.cpu_count)

        

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
          #  val_dataset["transcription"] = val_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', val_dataset["transcription"]).upper()
            val_dataset = val_dataset.map(self.transform_huggingface_dataset, num_proc = self.cpu_count)
            #self.test_dataset = SpeechRecognitionDataset(val_dataset,WAV2VEC2_ARGS,self.processor)
            self.test_dataset = val_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def transform_huggingface_dataset(self, batch: Dataset) -> Dataset:
        audio = batch["audio"]
        batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        batch["input_length"] = len(batch["input_values"])
        with self.processor.as_target_processor():
            batch["labels"] = self.processor(batch["transcription"]).input_ids
        return batch

    def data_collate(self, batch: Dataset) :
        input_features = [{"input_values": batch["input_values"]}]
        label_features = [{"input_ids": batch["labels"]}]
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )      
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

In [120]:
data = SpeechRecognitionDataModule(WAV2VEC2_ARGS, num_workers = 0)

CPU Count = 1


In [121]:
data.setup()

In Stage = Fit


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In Stage = Test


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
next(iter(data.val_dataloader()))

In [139]:
# from typing import Union, List, Dict
# import torch

# from dataclasses import dataclass, field
# from typing import Any, Dict, List, Optional, Union


# class DataCollatorCTCWithPadding:
#     """
#     Data collator that will dynamically pad the inputs received.
#     Args:
#         processor (:class:`~transformers.Wav2Vec2Processor`)
#             The processor used for proccessing the data.
#         padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
#             Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
#             among:
#             * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
#               sequence if provided).
#             * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
#               maximum acceptable input length for the model if that argument is not provided.
#             * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
#               different lengths).
#         max_length (:obj:`int`, `optional`):
#             Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
#         max_length_labels (:obj:`int`, `optional`):
#             Maximum length of the ``labels`` returned list and optionally padding length (see above).
#         pad_to_multiple_of (:obj:`int`, `optional`):
#             If set will pad the sequence to a multiple of the provided value.
#             This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
#             7.5 (Volta).
#     """

#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True
#     max_length: Optional[int] = None
#     max_length_labels: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
#     pad_to_multiple_of_labels: Optional[int] = None

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need
#         # different padding methods
#         input_features = [{"input_values": feature["input_values"]} for feature in features]
#         label_features = [{"input_ids": feature["labels"]} for feature in features]

#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt",
#         )
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 max_length=self.max_length_labels,
#                 pad_to_multiple_of=self.pad_to_multiple_of_labels,
#                 return_tensors="pt",
#             )

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         batch["labels"] = labels

#         return batch

In [204]:
class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.cpu_count = 1
        print(f"CPU Count = {self.cpu_count}")
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH).head(2)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(2)
        #songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.train_dataset = train_dataset.map(self.prepare_dataset,remove_columns = train_dataset.column_names)
            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.val_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)
            #common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

            self.test_dataset = val_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
        
    def prepare_dataset(self, batch):
        audio = batch["audio"]
        transcription = batch["transcription"]
        batch["input_values"] = audio["array"]
        batch["labels"] = transcription
        return batch
    

In [215]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        # self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(hparams.wav2vec2_model)
        # self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        # self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model)
        self.wav2vec2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
        self.seq2seq = AutoModelForSeq2SeqLM.from_pretrained(hparams.lm_model)
        #self.data_collator = DataCollatorCTCWithPadding()


    def forward(self, audio):
        print("entering forward step")
        print(audio[0].shape, len(audio))
        x = self.wav2vec2(audio[0])
        logits = self.seq2seq(x.view(x.size(0), -1))
        return logits

    def training_step(self, batch, batch_idx):
        print("entering training step")
        audio = batch["input_values"]
        labels = batch["labels"]
        logits = self(audio).logits
        loss = nn.CTCLoss(blank_id=0).forward(logits, labels)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch,batch_idx):
        print("entering validation step")
        print(batch)
        audio = batch["input_values"]
        labels = batch["labels"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio).logits
        loss = nn.CTCLoss(blank_id=0).forward(logits, labels)
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

# {'input': tensor([[-0.0004, -0.0008, -0.0004,  ..., -0.0038, -0.0031,  0.0000]],
#        device='cuda:0'), 'target': ['SOMETHING IS NOT THE SAME'], <DataKeys.METADATA: 'metadata'>: [{'sampling_rate': 16000}]}


    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     y_hat = self.model(x)
    #     loss = F.cross_entropy(y_hat, y)
    #     self.log("val_loss", loss)


In [216]:

import pytorch_lightning as pl
hparams = argparse.Namespace()
hparams.wav2vec2_model = 'facebook/wav2vec2-base'
hparams.lm_model = 't5-base' #'facebook/bart-large'
hparams.vocab_size = 10000
hparams.learning_rate = 0.001

model = Wav2SeqModel(hparams)
trainer = Trainer(max_epochs=1,devices=1, accelerator="gpu")
trainer.fit(model,SpeechRecognitionDataModule(WAV2VEC2_ARGS,num_workers=0))

# {'audio': [{'path': None, 'array': array([-0.05114746, -0.11273193, -0.09152222, ...,  0.2796936 ,
#         0.29998779,  0.20550537]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.14523315,  0.05508423, -0.13534546, ...,  0.01959229,
#         0.00973511, -0.00811768]), 'sampling_rate': 16000}], 'transcription': ["just a tryin' to survive", "don't go on me"], 'input_values':

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU

CPU Count = 1
In Stage = Fit


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                       | Params
--------------------------------------------------------
0 | wav2vec2 | Wav2Vec2ForCTC             | 94.4 M
1 | seq2seq  | T5ForConditionalGeneration | 222 M 
--------------------------------------------------------
317 M     Trainable params
0         Non-trainable params
317 M     Total params
1,269.199 Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 28 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


entering val data loader
entering validation step
{'input_values': [tensor([0.0020], device='cuda:0', dtype=torch.float64), tensor([0.0344], device='cuda:0', dtype=torch.float64), tensor([0.0226], device='cuda:0', dtype=torch.float64), tensor([0.0209], device='cuda:0', dtype=torch.float64), tensor([0.0529], device='cuda:0', dtype=torch.float64), tensor([0.0675], device='cuda:0', dtype=torch.float64), tensor([0.0718], device='cuda:0', dtype=torch.float64), tensor([0.0783], device='cuda:0', dtype=torch.float64), tensor([0.0889], device='cuda:0', dtype=torch.float64), tensor([0.0877], device='cuda:0', dtype=torch.float64), tensor([0.0993], device='cuda:0', dtype=torch.float64), tensor([0.1143], device='cuda:0', dtype=torch.float64), tensor([0.1077], device='cuda:0', dtype=torch.float64), tensor([0.0945], device='cuda:0', dtype=torch.float64), tensor([0.0991], device='cuda:0', dtype=torch.float64), tensor([0.0938], device='cuda:0', dtype=torch.float64), tensor([0.0682], device='cuda:0', dt

RuntimeError: Calculated padded input size per channel: (1). Kernel size: (10). Kernel size can't be greater than actual input size

In [159]:
# import librosa
# import torch
# from transformers import Wav2Vec2ForCTC

# def create_audio_tensor(audio_path):
#     audio, sample_rate = librosa.load(audio_path)
#     audio_tensor = torch.from_numpy(audio).float()
#     audio_tensor = audio_tensor.unsqueeze(0)
#     return audio_tensor

# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# audio_tensor = create_audio_tensor("notebooks/separated/mdx_extra/test_clip/vocals.wav")
# logits = model(audio_tensor).logits
# predictions = torch.argmax(logits, dim=-1)
# print(audio_tensor)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.1152, -0.1827, -0.1892,  ..., -0.2025, -0.2312, -0.1570]])


In [ ]:
class DALIDataset(pl.LightningDataModule):
    def __init__(self, batch_size: int = 4,
                  train_path :Optional[str] = None,
                    validation_path: Optional[str] = None,
                      model_backbone: pl.LightningModule = None,
                      args: TrainingArgs = WAV2VEC2_ARGS
                      ):
        
        super().__init__()
        self.train_path = train_path if validation_path is not None else args.TRAIN_FILE_PATH
        self.validation_path = validation_path if validation_path is not None else args.TEST_FILE_PATH
        self.model_backbone = model_backbone if model_backbone is not None else args.MODEL_BACKBONE

        def prepare_data(self):
            pass
        
        def setup(self):
            train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH) 
            validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
            songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
            audio_dataset = Dataset.from_dict(
                {"audio": list(songs_metadata["file_name"]),
                 "transcription": list(songs_metadata["transcription"])}).cast_column("audio", Audio(sampling_rate=16_000))
            audio_dataset["transcription"] = audio_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', audio_dataset["transcription"]).upper()
            audio_dataset = audio_dataset.train_test_split(test_size=0.2, shuffle=True)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule

class SpeechDataset(Dataset):
    def __init__(self, audio_paths, transcript_paths):
        self.audio_paths = audio_paths
        self.transcript_paths = transcript_paths

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, index):
        audio_path = self.audio_paths[index]
        transcript_path = self.transcript_paths[index]

        audio = torch.load(audio_path)
        transcript = torch.load(transcript_path)

        return audio, transcript

class SpeechDataModule(LightningDataModule):
    def __init__(self, audio_paths, transcript_paths, batch_size=16):
        super().__init__()
        self.audio_paths = audio_paths
        self.transcript_paths = transcript_paths
        self.batch_size = batch_size

    def prepare_data(self):
        # Download and extract the dataset
        pass

    def setup(self, stage=None):
        # Create the dataset
        self.dataset = SpeechDataset(self.audio_paths, self.transcript_paths)

        # Create the dataloader
        self.train_dataloader = DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4,
        )
        self.val_dataloader = DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=4,
        )

    def train_dataloader(self):
        return self.train_dataloader

    def val_dataloader(self):
        return self.val_dataloader